In [1]:
import numpy as np
import math
from scipy.stats import norm

## 1. How to get the implied volatility of each option by using Newton-Raphson method

In [2]:
def BS_price(option_type, s, k, r, q, t, sigma):
    
    stdnorm = norm(loc=0, scale=1)
    
    if option_type == "Call":
        o_type = 1
    elif option_type == "Put":
        o_type = -1
    
    d1 = (np.log(s/ k) + (r - q + 0.5 * sigma**2) * t) / (sigma * np.sqrt(t))
    d2 = d1 - sigma * np.sqrt(t)
    
    nd1 = stdnorm.cdf(o_type * d1)
    nd2 = stdnorm.cdf(o_type * d2)
    
    price = o_type * (s * np.exp(-q * t) * nd1 - k * np.exp(-r * t) * nd2)
    
    return price

In [3]:
def Newton_Raphson(option_type, s0, k, r, q, t, market_price):
    
    max_iter = 1000
    tol = 0.000001
    v0 = 0.15   #initial volatility = 15%
    
    if option_type == 'Call':
        if s0 - k * np.exp(-r * t) > market_price:
            print("The market price of option is out of arbitrage lower bound")
        if market_price > s0:
            print("The market price of option is out of arbitrage upper bound")
    
    elif option_type == 'Put':
        if k * np.exp(-r * t) - s0 > market_price:
            print("The market price of option is out of arbitrage lower bound")
        if market_price > k * np.exp(-r * t):
            print("The market price of option is out of arbitrage upper bound")
    
    for i in range(max_iter):
        
        f0 = BS_price(option_type, s0, k, r, q, t, v0) - market_price
        d1 = (np.log(s0/ k) + (r - q + 0.5 * v0**2) * t) / (v0 * np.sqrt(t))
        pdf_d1 = np.exp(-(d1**2)/2) / np.sqrt(2 * math.pi)

        vega = s0 * pdf_d1 * np.exp(-q * t) * np.sqrt(t)
        
        if vega <= 0:
            print("vega is zero or negative.")
            break
        
        v = v0 - f0/vega
        err = v - v0
        
        if abs(f0) < tol or abs(err) < tol:
            return v
        else:
            v0 = v

In [4]:
option_type = "Call"
s0 = 273.55
k = 257.5
r = 0.0155
q = 0
t = 10 / 365
market_price = 16.15

im_vol = Newton_Raphson(option_type, s0, k, r, q, t, market_price)
print("The implied volatility of KOSPI200 C 201910 257.5 is : ", im_vol)

The market price of option is out of arbitrage lower bound
vega is zero or negative.
The implied volatility of KOSPI200 C 201910 257.5 is :  None


In [5]:
option_type = "Call"
s0 = 273.55
k = 260.0
r = 0.0155
q = 0
t = 10/365
market_price = 14

im_vol = Newton_Raphson(option_type, s0, k, r, q, t, market_price)
print("The implied volatility of KOSPI200 C 201910 260.0 is : ", im_vol)

The implied volatility of KOSPI200 C 201910 260.0 is :  0.21829291058680864


## 2. Solve the n-simultaneous equations by using Thomas algorithm

In [6]:
arr = np.array([[1, 2, 0, 0, 0, 0],
                [-2, 1, 1, 0, 0, 0],
                [0, 0.5, 1, 0.5, 0, 0],
                [0, 0, -2, 1, 0.5, 0],
                [0, 0, 0, 1, 0.2, 0.3],
                [0, 0, 0, 0, 1, 1]])

d = np.array([2, -1, 0.5, 2.3, -5, 0.25])

In [7]:
def Thomas_algorithm(arr, d):    
    n = len(d)
    a_prime = np.zeros(n)
    d_prime = np.zeros(n)
    x = np.zeros(n)
    
    a_prime[0] = arr[0][0]
    d_prime[0] = d[0]
    
    for i in range(1, n):
        a_prime[i] = arr[i][i] - arr[i-1][i] * (arr[i][i-1] / a_prime[i-1]) 
        d_prime[i] = d[i] - d_prime[i-1] * (arr[i][i-1] / a_prime[i-1])
    
    x[n-1] = d_prime[n-1] / a_prime[n-1]
    
    for j in range(n-2,-1,-1):
        x[j] = (d_prime[j]/a_prime[j]) - (arr[j][j+1]/a_prime[j]) * x[j+1]
    
    return x

In [8]:
print(Thomas_algorithm(arr, d))

[  1.59609375   0.20195312   1.99023438  -3.18242188  18.92578125
 -18.67578125]
